In [ ]:
# ============================================================
# Core scientific stack
# ============================================================
import numpy as np
import matplotlib.pyplot as plt


# ============================================================
# Optional local circular coordinates
# ============================================================
from dreimac import CircularCoords


# ============================================================
# Persistent homology
# ============================================================
from ripser import ripser
from persim import plot_diagrams


# ============================================================
# circle_bundles core API + analysis tools
# ============================================================
from circle_bundles.api import build_bundle
from circle_bundles.base_covers import MetricBallCover
from circle_bundles.metrics import (
    S1AngleMetric as s1_metric,
    RP1AngleMetric as rp1_metric
)

from circle_bundles.analysis.local_analysis import (
    get_local_rips, 
    plot_local_rips, 
)

# ============================================================
# Optical flow data processing + features
# ============================================================

from circle_bundles.optical_flow.contrast import get_predominant_dirs, get_lifted_predom_dirs


# ============================================================
# Visualization utilities
# ============================================================
from circle_bundles.optical_flow.patch_viz import make_patch_visualizer
from circle_bundles.viz.thumb_grids import show_data_vis
from circle_bundles.viz.lattice_vis import lattice_vis
from circle_bundles.viz.circle_vis import circle_vis, circle_vis_grid
from circle_bundles.viz.angles import compare_angle_pairs
from circle_bundles.viz.pca_vis import show_pca
from circle_bundles.viz.bundle_dash import show_bundle_vis

# ============================================================
# Attach optional BundleResult visualization methods
# ============================================================
from circle_bundles.bundle import attach_bundle_viz_methods
attach_bundle_viz_methods()


# Load The Dataset

In [ ]:
import pickle
import pandas as pd

#Load the dataset of binary step-edge circles extracted from K_50_60

file_path = '.../K_50_60_Circles.pkl'  #path to clustered data

with open(file_path, 'rb') as f:
    [data, C_circ] =pickle.load(f)

#Compute the predominant direction and directionality for each patch
predom_dirs, ratios = get_predominant_dirs(data)

#Create a visualizer for optical flow patches
patch_vis = make_patch_visualizer()

print(f'Sample contains {len(data)} high-contrast optical flow patches.')
print(f'A total of {len(C_circ)} clusters.')


# Preliminary Analysis

In [ ]:
#Show a sample of patches from a circular cluster

j = 17   #Choose a circular cluster
fig = show_data_vis(data[C_circ[j]], patch_vis, max_samples = 30)
plt.show()

In [ ]:
#View PCA of the full dataset colored according to assigned circular cluster
show_pca(data, U = C_circ, set_cmap = 'turbo', max_points = 20000)


In [ ]:
#Show an interactive visualization of the projection map to RP1
#colored by assigned fiber coordinate

app = show_bundle_vis(
    base_points = predom_dirs,
    data = data,
    base_metric = rp1_metric(),
    max_samples = 10000)


In [ ]:
#Show correlations between circular coordinate and predominant direction on each circular component

#Run circular coordinates on the patches in each cluster 
to_show = [9, 11, 15]
n_landmarks = 100
prime = 17

circle_datasets = []
all_angles = []
titles = []

for j in to_show:
    indices = C_circ[j]
    circle_datasets.append(data[indices])
    cc = CircularCoords(data[indices], n_landmarks, prime=prime)
    all_angles.append(cc.get_coordinates())
    titles.append(f'Component {j}')

fig, axes = circle_vis_grid(
    circle_datasets,
    all_angles,
    patch_vis,
    titles=titles,
    per_circle=8,
    circle_radius=1.0,
    extent_factor=1.2,
    circle_zoom=0.13,
    circle_linewidth=1.0,
    circle_color="black",
    n_cols=3,
    title_fontsize=16,
    figsize_per_panel=5,
    fig_dpi=150,
)
plt.show()

#Show correlations between circular coordinate and predominant direction on each circular component
angle_arrays = []
to_compare = []
labels = []
empty_titles = []

for j, ang_arr in enumerate(all_angles):
    angle_arrays.append(ang_arr)
    indices = C_circ[to_show[j]]
    predoms = predom_dirs[indices]
    angle_arrays.append(predoms)
    if j == 0:
        labels.append(f'Circular Coordinate')
    else:
        labels.append('')
    labels.append(f'Predominant Flow Axis')
    empty_titles.append('')
    to_compare.append([2*j+1,2*j])


fig = compare_angle_pairs(
    angle_arrays,
    to_compare,
    x_range = (0,np.pi),
    labels = labels,
    align = False,
    s = 1.0,
    fontsize = 14,
    ncols = 3,
    titles = empty_titles,
    titlesize = 20,
    show_metrics = False)
plt.show()




## Lifting The Step Edge Circles

In [ ]:
#Compute lifted predominant angles in S1

lifted_base_points = get_lifted_predom_dirs(data, eps=1e-12)
lifted_angles = np.arctan2(lifted_base_points[:,1], lifted_base_points[:,0]) % (2*np.pi)

n_landmarks = 16
landmarks = np.linspace(0, 2*np.pi,n_landmarks, endpoint= False)
overlap = 1.99
radius = overlap* np.pi/n_landmarks

lifted_cover = MetricBallCover(lifted_angles, landmarks, radius, metric = s1_metric())

lifted_cover_data = lifted_cover.build()

lifted_summ = lifted_cover.summarize(plot = True)

In [ ]:
#Compute local 1-D persistence

to_view = [3,6,14]
fiber_ids, dense_idx_list, rips_list = get_local_rips(
    data,
    lifted_cover.U,
    p_values=None,
    to_view=to_view,
    maxdim=1,
    n_perm=500,
    random_state=None,
)

fig, axes = plot_local_rips(
    fiber_ids,
    rips_list,
    n_cols=3,
    titles='default',
    font_size=20,
)

In [ ]:
#Compute local trivializations and characteristic classes
lifted_bundle = build_bundle(
    data,
    lifted_cover,
#    CircularCoords_cls=CircularCoords,    #optionally use sparse cc's
    show=True)


In [ ]:
#Get a global toroidal coordinate system
triv_result = lifted_bundle.get_global_trivialization()
fiber_angles = triv_result.F
print('Global coordinates computed.')

In [ ]:
#Show a sample of coordinatized data

coords = np.array([lifted_angles, fiber_angles]).T
fig = lattice_vis(data, coords, patch_vis, 
                             per_row=7,
                              per_col = 9,
                             figsize=22)
plt.show()


In [ ]:
#Show an interactive visualization of the bundle structure
app = lifted_bundle.show_bundle(colors = fiber_angles)


In [ ]:
#Show an interactive visualization of the original projection map with data 
#colored by assigned fiber coordinate

app = show_bundle_vis(
    base_points = predom_dirs,
    data = data,
    base_metric = rp1_metric(),
    colors = fiber_angles,
    max_samples = 10000)
